# Multiple-variable-house-price-prediction-linear-regression

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn import linear_model

In [2]:
df = pd.read_csv("/kaggle/input/housing-prices-dataset/Housing.csv")
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


# data preprocessing 

In [3]:
#Check data types (numeric, object, etc.)

#See if there are any NaN (missing) values.


df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB


In [4]:
binary_columns = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
df[binary_columns] = df[binary_columns].replace({'yes': 1, 'no': 0}).infer_objects(copy=False)
df

<ipython-input-4-dcbfa2fcd33b>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[binary_columns] = df[binary_columns].replace({'yes': 1, 'no': 0}).infer_objects(copy=False)


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,furnished
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,furnished
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,semi-furnished
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,furnished
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,1,0,1,0,0,2,0,unfurnished
541,1767150,2400,3,1,1,0,0,0,0,0,0,0,semi-furnished
542,1750000,3620,2,1,1,1,0,0,0,0,0,0,unfurnished
543,1750000,2910,3,1,1,0,0,0,0,0,0,0,furnished


In [5]:
# One-hot encode 'furnishingstatus'
df = pd.get_dummies(df, columns=['furnishingstatus'], drop_first=True)
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,False,False
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,False,False
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,True,False
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,False,False
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,1,0,1,0,0,2,0,False,True
541,1767150,2400,3,1,1,0,0,0,0,0,0,0,True,False
542,1750000,3620,2,1,1,1,0,0,0,0,0,0,False,True
543,1750000,2910,3,1,1,0,0,0,0,0,0,0,False,False


In [6]:
# Convert boolean columns to integers
df = df.astype(int)
df

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus_semi-furnished,furnishingstatus_unfurnished
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,0,0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,0,0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1,0
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,0,0
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,1,0,1,0,0,2,0,0,1
541,1767150,2400,3,1,1,0,0,0,0,0,0,0,1,0
542,1750000,3620,2,1,1,1,0,0,0,0,0,0,0,1
543,1750000,2910,3,1,1,0,0,0,0,0,0,0,0,0


In [7]:
#train model
X = df.drop('price', axis=1)   # All columns except 'price'
y = df['price']   
model = linear_model.LinearRegression()
model.fit(X,y)

LinearRegression()

In [8]:
#train test split
from sklearn.model_selection import train_test_split 

In [9]:
X_train , X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [10]:
model.fit(X_train,y_train)

LinearRegression()

In [11]:
model.predict(X_test)

array([2168029.31334307, 5962716.04292312, 2775938.73720624,
       3672202.51444004, 4846555.46252768, 3011533.57658358,
       4768129.93767915, 4560219.42929114, 4479722.78315198,
       4343181.13098475, 6382629.2729013 , 2693129.0218787 ,
       3434438.66838395, 7613543.27162194, 6348155.62540099,
       4692145.23171007, 4343653.035144  , 6106297.26541753,
       2956826.44619443, 3439426.02783305, 6776412.88029813,
       4172944.06985739, 3839870.13609789, 3792388.74567994,
       5329331.03321387, 6546669.6072372 , 4530719.88597749,
       4852511.22173234, 4032175.20551081, 4361113.79454207,
       4206321.51601666, 6488211.25475788, 4615463.89717606,
       2836877.65876592, 7702003.38498922, 5775931.95482503,
       3630336.19464159, 6381322.11395872, 3398074.20260868,
       3640703.45123094, 5599375.42780545, 7326971.429673  ,
       7552205.3525506 , 4845629.3594185 , 3564714.05963993,
       3761774.15721315, 7301186.78923   , 4222880.74063613,
       3089910.85210898,

In [12]:
y_test

528    2275000
181    5215000
517    2450000
440    3234000
298    4200000
        ...   
213    4893000
197    4956000
433    3290000
522    2380000
364    3703000
Name: price, Length: 109, dtype: int64

In [13]:
model.score(X_test,y_test)

0.7501162765706997